# Preprocess Gene Annotations

This notebook creates a table of gene annotations by:
1. Querying Biomart for all Ensembl IDs in the database
2. Querying MyGene for annotation about those IDs
3. Querying Ensembl for the most recent Ensembl release for each ID
4. Building a permalink to the Ensembl archive page for each ID

This gene annotation table is read in by `agoradataprocessing/process.py` to be used in the `gene_info` transformation. 

***Note:*** *This notebook is exploratory and should eventually be converted to a Python script that is run through an automated process.*

## Installation requirements

#### Linux / Windows / Mac

Install R: https://cran.r-project.org/

Install Python and agora-data-tools following the instructions in this repository's README. This notebook assumes it is being run from the same `pipenv` virtual environment as agora-data-tools. 

Then install the following packages using `pip`:
```
pip install rpy2 mygene
```

#### Note for Macs with M1 chips (2020 and newer)

Install as above, but make sure that your R installation is the arm64 version (R-4.X.X-arm64.pkg) so that the architecture matches what pip is using. 
You may also need to install an older version of `rpy2` on the Mac:
```
pip install rpy2==3.5.12
```

In [ ]:
from rpy2.robjects import r
import pandas as pd
import mygene
import numpy as np
import requests
import agoradatatools.etl.utils as utils
import agoradatatools.etl.extract as extract
import preprocessing_utils

r(
    'if (!require("BiocManager", character.only = TRUE)) { install.packages("BiocManager") }'
)
r('if (!require("biomaRt")) { BiocManager::install("biomaRt") }')

r.library("biomaRt")

ensembl_ids_filename = "../../output/ensembl_id_list.txt"
archive_filename = "../../output/ensembl_archive_list.csv"
config_filename = "../../../../config.yaml"

# Part 1: Get gene annotation data

## [Deprecated] Query Biomart for a list of all Ensembl IDs in the database of human genes. 

Here we use the R library `biomaRt`. There is no canonical Python library with the features we need for this script. 

*We no longer get all genes from BioMart, so this section is unused. The code is here in case we need it again.*

In [ ]:
"""
ensembl_ids_df = preprocessing_utils.r_query_biomart()
ensembl_ids_df = preprocessing_utils.filter_hasgs(
    df=ensembl_ids_df, chromosome_name_column="chromosome_name"
)
print(str(ensembl_ids_df.shape[0]) + " genes remaining after HASG filtering.")
"""

## Get Ensembl IDs from data sets that will be processed by agora-data-tools

Loop through all data sets in the config file to get all Ensembl IDs used in every data set. Exclude `gene_metadata` since that's the file we are building, and `druggability` since that data is deprecated.

In [3]:
file_ensembl_list = preprocessing_utils.get_all_adt_ensembl_ids(
    config_filename=config_filename,
    exclude_files=["gene_metadata", "druggability"],
    token=None,
)
print("")
print(str(len(file_ensembl_list)) + " Ensembl IDs found.")
print(file_ensembl_list[0:5])


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.6.0) is available. Your version (4.0.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.6.0 release notes

https://python-docs.synapse.org/news/



Welcome, Jaclyn Beck!



INFO:synapseclient_default:Welcome, Jaclyn Beck!



Found 19 files:
genes_biodomains:	syn44151254.5
neuropath_regression_results:	syn22017882.5
proteomics:	syn18689335.4
proteomics_tmt:	syn35221005.2
proteomics_srm:	syn52579640.4
target_exp_validation_harmonized:	syn24184512.9
metabolomics:	syn26064497.1
gene_metadata:	syn25953363.13
igap:	syn12514826.5
eqtl:	syn12514912.3
diff_exp_data:	syn27211942.1
target_list:	syn12540368.51
median_expression:	syn27211878.2
druggability:	syn13363443.11
tep_adi_info:	syn51942280.3
team_info:	syn12615624.18
team_member_info:	syn12615633.19
overall_scores:	syn25575156.13
networks:	syn11685347.1

genes_biodomains has 591 NaN Ensembl IDs

35858 Ensembl IDs found.
['ENSG00000151650', 'ENSG00000168268', 'ENSG00000186310', 'ENSG00000204616', 'ENSG00000158467']


Create a data frame with these IDs so it can be merged with the MyGene query results below.

In [4]:
ensembl_ids_df = pd.DataFrame({"ensembl_gene_id": file_ensembl_list})

""" Removed due to no longer getting genes from BioMart, but saving code
# Add Ensembl IDs that are in the files but not in the biomart result
missing = set(file_ensembl_list) - set(ensembl_ids_df["ensembl_gene_id"])
print(
    str(len(missing))
    + " genes from the data files are missing from Biomart results and will be added."
)

missing_df = pd.DataFrame({"ensembl_gene_id": list(missing), "chromosome_name": ""})
ensembl_ids_df = pd.concat([ensembl_ids_df, missing_df])
"""

ensembl_ids_df = ensembl_ids_df.dropna(subset=["ensembl_gene_id"])
print(len(ensembl_ids_df))

35858


In [5]:
# Write to a file to save the list of IDs
ensembl_ids_df.to_csv(
    path_or_buf=ensembl_ids_filename, sep="\t", header=False, index=False
)

## Get info on each gene from mygene

In [6]:
mg = mygene.MyGeneInfo()

mygene_output = mg.getgenes(
    ensembl_ids_df["ensembl_gene_id"],
    fields=["symbol", "name", "summary", "type_of_gene", "alias"],
    as_dataframe=True,
)

mygene_output.index.rename("ensembl_gene_id", inplace=True)
mygene_output.head()

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-12000...
INFO:biothings.client:done.
INFO:biothings.client:querying 12001-13000...
INFO:biothings.client:done.
INFO:biothings.client:querying 13001-14000...
INFO:biothings

,_id,_version,alias,name,summary,symbol,type_of_gene,notfound
ensembl_gene_id,,,,,,,,
ENSG00000151650,27287,1.0,"[HPX42B, NA88A, VENTX2]",VENT homeobox,This gene encodes a member of the Vent family ...,VENTX,protein-coding,NaN
ENSG00000168268,64943,1.0,NaN,5'-nucleotidase domain containing 2,Predicted to enable 5'-nucleotidase activity. ...,NT5DC2,protein-coding,NaN
ENSG00000186310,4675,1.0,"[MB20, NPL3]",nucleosome assembly protein 1 like 3,This gene is intronless and encodes a member o...,NAP1L3,protein-coding,NaN
ENSG00000204616,11074,1.0,"[C6orf13, HCG1, HCGI, RNF]",tripartite motif containing 31,This gene encodes a protein that functions as ...,TRIM31,protein-coding,NaN
ENSG00000158467,23382,1.0,"[ADOHCYASE3, IRBIT2]",adenosylhomocysteinase like 2,The protein encoded by this gene acts as a hom...,AHCYL2,protein-coding,NaN


In [7]:
print("Annotations found for " + str(sum(mygene_output["notfound"].isna())) + " genes.")
print(
    "No annotations found for "
    + str(sum(mygene_output["notfound"] == True))
    + " genes."
)

Annotations found for 34655 genes.
No annotations found for 1206 genes.


# Part 2: Clean the data

## Join and standardize columns / values

For consistency with the `agora-data-tools` transform process, this uses the etl standardize functions.

In [27]:
gene_table_merged = pd.merge(
    left=ensembl_ids_df,
    right=mygene_output,
    how="left",
    on="ensembl_gene_id",
    validate="many_to_many",
)

gene_table_merged = utils.standardize_column_names(gene_table_merged)
gene_table_merged = utils.standardize_values(gene_table_merged)

print(gene_table_merged.shape)
gene_table_merged.head()

(35861, 9)


,ensembl_gene_id,_id,_version,alias,name,summary,symbol,type_of_gene,notfound
0,ENSG00000151650,27287,1.0,"[HPX42B, NA88A, VENTX2]",VENT homeobox,This gene encodes a member of the Vent family ...,VENTX,protein-coding,NaN
1,ENSG00000168268,64943,1.0,NaN,5'-nucleotidase domain containing 2,Predicted to enable 5'-nucleotidase activity. ...,NT5DC2,protein-coding,NaN
2,ENSG00000186310,4675,1.0,"[MB20, NPL3]",nucleosome assembly protein 1 like 3,This gene is intronless and encodes a member o...,NAP1L3,protein-coding,NaN
3,ENSG00000204616,11074,1.0,"[C6orf13, HCG1, HCGI, RNF]",tripartite motif containing 31,This gene encodes a protein that functions as ...,TRIM31,protein-coding,NaN
4,ENSG00000158467,23382,1.0,"[ADOHCYASE3, IRBIT2]",adenosylhomocysteinase like 2,The protein encoded by this gene acts as a hom...,AHCYL2,protein-coding,NaN


## Fix alias field

Fix `NaN` values in the `alias` field and make sure every alias value is a list, not a string.

In [28]:
# NaN or NULL alias values become empty lists
gene_table_merged["alias"] = gene_table_merged["alias"].apply(
    lambda cell: cell if cell is not np.NaN else []
)

# Some alias values are a single string, not a list. Turn them into lists here.
gene_table_merged["alias"] = gene_table_merged["alias"].apply(
    lambda cell: cell if isinstance(cell, list) else [cell]
)


# Some alias values are lists of lists or have duplicate values
def flatten(row):
    flattened = []
    for item in row:
        if isinstance(item, list):
            flattened = flattened + item
        else:
            flattened.append(item)
    return flattened


gene_table_merged["alias"] = gene_table_merged["alias"].apply(
    lambda row: list(set(flatten(row)))
)

## Remove duplicate Ensembl IDs from the list. 

Duplicates in the list typically have the same Ensembl ID but different gene symbols. This usually happens when a single Ensembl ID maps to multiple Entrez IDs in the NCBI database. There's not a good way to reconcile this, so we first check for entries whose `symbol` is something other than "LOC#######", and designate that entry as the main row. If there are multiple or zero entries meeting that criteria, we just use the first entry in the list for each ensembl ID and discard the rest, which is what the Agora front end does. The gene symbols of duplicate rows are then added as aliases to the matching unique row.

In [29]:
# duplicated() will return true if the ID is a duplicate and is not the first one to appear the list.
dupes = gene_table_merged["ensembl_gene_id"].duplicated()
dupe_vals = gene_table_merged[dupes]

# Rows with duplicated Ensembl IDs
all_duplicated = gene_table_merged.loc[
    gene_table_merged["ensembl_gene_id"].isin(dupe_vals["ensembl_gene_id"])
]
all_duplicated

,ensembl_gene_id,_id,_version,alias,name,summary,symbol,type_of_gene,notfound
19626,ENSG00000249738,285626,1.0,[],uncharacterized LOC285626,NaN,LOC285626,ncRNA,NaN
19627,ENSG00000249738,105377683,1.0,[],uncharacterized LOC105377683,NaN,LOC105377683,ncRNA,NaN
24698,ENSG00000276387,3802,1.0,"[CD158A, NKAT1, KIR2DL3, KIR-K64, NKAT-1, p58....","killer cell immunoglobulin like receptor, two ...",Killer cell immunoglobulin-like receptors (KIR...,KIR2DL1,protein-coding,NaN
24699,ENSG00000276387,124900571,1.0,[],killer cell immunoglobulin-like receptor 2DS1,NaN,LOC124900571,protein-coding,NaN
29514,ENSG00000230373,100133220,1.0,[GOLGA6L3],"golgin A6 family like 3, pseudogene",NaN,GOLGA6L3P,pseudo,NaN
29515,ENSG00000230373,642402,1.0,[GOLGA6L21P],"golgin A6 family like 17, pseudogene",NaN,GOLGA6L17P,pseudo,NaN


In [ ]:
keep_df = gene_table_merged.drop(all_duplicated.index)

# For each duplicated Ensembl ID, collapse to 1 row and append that row to keep_df
for ens_id in set(all_duplicated["ensembl_gene_id"]):
    group = all_duplicated.loc[all_duplicated["ensembl_gene_id"] == ens_id].copy(
        deep=True
    )
    # Put any entries with symbols that aren't "LOC#####" at the top of the data frame
    matches = group["symbol"].str.startswith("LOC") == False
    group = pd.concat([group.loc[matches], group.loc[matches == False]]).reset_index(
        drop=True
    )

    # Add all duplicate symbols and their aliases to the alias field of the first entry
    for row in group.index[1:]:
        group.at[group.index[0], "alias"].append(group["symbol"][row])
        if len(group.at[row, "alias"]) > 0:
            group.at[group.index[0], "alias"] = (
                group.at[group.index[0], "alias"] + group["alias"][row]
            )

    # Make sure we didn't add duplicate aliases
    group.at[group.index[0], "alias"] = list(set(group.at[group.index[0], "alias"]))

    # Keep the first row only, which now has all the aliases
    keep_df = pd.concat([keep_df, group.iloc[0].to_frame().T], ignore_index=True)

print(
    str(len(all_duplicated.drop_duplicates("ensembl_gene_id")))
    + " duplicated genes have been processed."
)
gene_table_merged = keep_df.reset_index(drop=True)
gene_table_merged.tail(n=10)

3 duplicated genes have been processed.


,ensembl_gene_id,_id,_version,alias,name,summary,symbol,type_of_gene,notfound
35848,ENSG00000085998,55624,1.0,"[RP76, LGMDR15, LGMD2O, gnT-I.2, GNTI.2, GnT I...",protein O-linked mannose N-acetylglucosaminylt...,This gene encodes a type II transmembrane prot...,POMGNT1,protein-coding,NaN
35849,ENSG00000285081,ENSG00000285081,1.0,[],NaN,NaN,NaN,NaN,NaN
35850,ENSG00000126822,26030,1.0,"[ARHGEF43, KIAA0599]",pleckstrin homology and RhoGEF domain containi...,Predicted to enable guanyl-nucleotide exchange...,PLEKHG3,protein-coding,NaN
35851,ENSG00000187240,79659,1.0,"[DHC2, hdhc11, DNCH2, SRTD3, SRPS2B, ATD3, DHC...",dynein cytoplasmic 2 heavy chain 1,This gene encodes a large cytoplasmic dynein p...,DYNC2H1,protein-coding,NaN
35852,ENSG00000101470,7125,1.0,"[CMYP15, CMYO15, CFAP85, FAP85, MYONRI]","troponin C2, fast skeletal type","Troponin (Tn), a key protein complex in the re...",TNNC2,protein-coding,NaN
35853,ENSG00000241472,100506994,1.0,[],PTPRG antisense RNA 1,NaN,PTPRG-AS1,ncRNA,NaN
35854,ENSG00000133106,94240,1.0,[BRESI1],epithelial stromal interaction 1,The protein encoded by this gene has been show...,EPSTI1,protein-coding,NaN
35855,ENSG00000230373,100133220,1.0,"[GOLGA6L3, GOLGA6L21P, GOLGA6L17P]","golgin A6 family like 3, pseudogene",NaN,GOLGA6L3P,pseudo,NaN
35856,ENSG00000249738,285626,1.0,[LOC105377683],uncharacterized LOC285626,NaN,LOC285626,ncRNA,NaN
35857,ENSG00000276387,3802,1.0,"[CD158A, NKAT1, KIR2DL3, LOC124900571, KIR-K64...","killer cell immunoglobulin like receptor, two ...",Killer cell immunoglobulin-like receptors (KIR...,KIR2DL1,protein-coding,NaN


# Part 3: Create Ensembl archive permalinks

## Get a table of Ensembl archive URLs

This is where we need to use the R biomaRt library specifically, instead of any of the available Python interfaces to Biomart, to get a table of Ensembl release versions and their corresponding archive URLs. 

In [12]:
archive_df = r.listEnsemblArchives()
archive_df.to_csvfile(path=archive_filename, row_names=False, quote=False)

print(archive_df)

             name     date                                 url version
1  Ensembl GRCh37 Feb 2014          https://grch37.ensembl.org  GRCh37
2     Ensembl 113 Oct 2024 https://oct2024.archive.ensembl.org     113
3     Ensembl 112 May 2024 https://may2024.archive.ensembl.org     112
4     Ensembl 111 Jan 2024 https://jan2024.archive.ensembl.org     111
5     Ensembl 110 Jul 2023 https://jul2023.archive.ensembl.org     110
6     Ensembl 109 Feb 2023 https://feb2023.archive.ensembl.org     109
7     Ensembl 108 Oct 2022 https://oct2022.archive.ensembl.org     108
8     Ensembl 107 Jul 2022 https://jul2022.archive.ensembl.org     107
9     Ensembl 106 Apr 2022 https://apr2022.archive.ensembl.org     106
10    Ensembl 105 Dec 2021 https://dec2021.archive.ensembl.org     105
11    Ensembl 104 May 2021 https://may2021.archive.ensembl.org     104
12    Ensembl 103 Feb 2021 https://feb2021.archive.ensembl.org     103
13    Ensembl 102 Nov 2020 https://nov2020.archive.ensembl.org     102
14    

## Query Ensembl for each gene's version

Ensembl's REST API can only take 1000 genes at once, so this is looped to query groups of 1000. 

In [13]:
versions = preprocessing_utils.query_ensembl_version_api(
    ensembl_ids=gene_table_merged["ensembl_gene_id"].tolist()
)

versions.tail()

Querying genes 1 - 1000
Querying genes 1001 - 2000
Querying genes 2001 - 3000
Querying genes 3001 - 4000
Querying genes 4001 - 5000
Querying genes 5001 - 6000
Querying genes 6001 - 7000
Querying genes 7001 - 8000
Querying genes 8001 - 9000
Querying genes 9001 - 10000
Querying genes 10001 - 11000
Querying genes 11001 - 12000
Querying genes 12001 - 13000
Querying genes 13001 - 14000
Querying genes 14001 - 15000
Querying genes 15001 - 16000
Querying genes 16001 - 17000
Querying genes 17001 - 18000
Querying genes 18001 - 19000
Querying genes 19001 - 20000
Querying genes 20001 - 21000
Querying genes 21001 - 22000
Querying genes 22001 - 23000
Querying genes 23001 - 24000
Querying genes 24001 - 25000
Querying genes 25001 - 26000
Querying genes 26001 - 27000
Querying genes 27001 - 28000
Querying genes 28001 - 29000
Querying genes 29001 - 30000
Querying genes 30001 - 31000
Querying genes 31001 - 32000
Querying genes 32001 - 33000
Querying genes 33001 - 34000
Querying genes 34001 - 35000
Queryin

,assembly,peptide,possible_replacement,release,latest,type,id,version,is_current
35853,GRCh38,None,[],113,ENSG00000241472.9,Gene,ENSG00000241472,9,1
35854,GRCh38,None,[],113,ENSG00000133106.15,Gene,ENSG00000133106,15,1
35855,GRCh38,None,[],113,ENSG00000230373.9,Gene,ENSG00000230373,9,1
35856,GRCh38,None,[],113,ENSG00000249738.11,Gene,ENSG00000249738,11,1
35857,GRCh38,None,[],113,ENSG00000276387.4,Gene,ENSG00000276387,4,1


In [14]:
versions.groupby("release").size()

release
100       21
101        8
102       16
103       12
104       17
105       10
106       35
107       12
108        4
109        4
110       11
111       52
112      354
113    34303
80        21
81         2
82        10
84       673
87        61
89        20
91        67
93        50
95        33
96        31
97        17
98         9
99         5
dtype: int64

In [15]:
# Check that all IDs are the same between the result and the gene table
print(len(versions["id"]))
print(len(gene_table_merged))
print(
    all(versions["id"].isin(gene_table_merged["ensembl_gene_id"]))
    and all(gene_table_merged["ensembl_gene_id"].isin(versions["id"]))
)

35858
35858
True


In [16]:
# Make sure everything is GRCh38, not GRCh37
all(versions["assembly"] == "GRCh38")

True

## Create permalinks based on archive version

**Not all of these versions have an archive.** We can go back to the closest previous archive for these but the link isn't guaranteed to work.

In [17]:
archive_table = pd.read_csv(archive_filename)

# Remove GRCh37 from the archive list
archive_table = archive_table[archive_table["version"] != "GRCh37"].reset_index()

archive_table["numeric_version"] = archive_table["version"].astype(int)


def closest_release(release, archive_table):
    if release in archive_table:
        return release

    return max([V for V in archive_table["numeric_version"] if V <= release])

In [18]:
versions["closest_release"] = 0

releases = versions["release"].drop_duplicates().astype(int)

# Only have to call closest_release once per version, instead of >70k times
for release in releases:
    versions.loc[versions["release"] == str(release), "closest_release"] = (
        closest_release(release, archive_table)
    )

versions.groupby("closest_release").size()

closest_release
80       985
98         9
99         5
100       21
101        8
102       16
103       12
104       17
105       10
106       35
107       12
108        4
109        4
110       11
111       52
112      354
113    34303
dtype: int64

In [19]:
versions["permalink"] = ""

for i in versions.index:
    match = archive_table["numeric_version"] == versions.at[i, "closest_release"]
    url = archive_table.loc[match, "url"].to_string(index=False)
    if len(url) > 0:
        versions.at[i, "permalink"] = (
            url + "/Homo_sapiens/Gene/Summary?db=core;g=" + versions.at[i, "id"]
        )

versions.head()

,assembly,peptide,possible_replacement,release,latest,type,id,version,is_current,closest_release,permalink
0,GRCh38,None,[],113,ENSG00000151650.8,Gene,ENSG00000151650,8,1,113,https://oct2024.archive.ensembl.org/Homo_sapie...
1,GRCh38,None,[],113,ENSG00000168268.11,Gene,ENSG00000168268,11,1,113,https://oct2024.archive.ensembl.org/Homo_sapie...
2,GRCh38,None,[],113,ENSG00000186310.10,Gene,ENSG00000186310,10,1,113,https://oct2024.archive.ensembl.org/Homo_sapie...
3,GRCh38,None,[],113,ENSG00000204616.11,Gene,ENSG00000204616,11,1,113,https://oct2024.archive.ensembl.org/Homo_sapie...
4,GRCh38,None,[],113,ENSG00000158467.17,Gene,ENSG00000158467,17,1,113,https://oct2024.archive.ensembl.org/Homo_sapie...


In [20]:
versions[versions["closest_release"] < 100].head()

,assembly,peptide,possible_replacement,release,latest,type,id,version,is_current,closest_release,permalink
67,GRCh38,None,[],84,ENSG00000265108.1,Gene,ENSG00000265108,1,,80,https://may2015.archive.ensembl.org/Homo_sapie...
68,GRCh38,None,[],80,ENSG00000280803.1,Gene,ENSG00000280803,1,,80,https://may2015.archive.ensembl.org/Homo_sapie...
111,GRCh38,None,[],84,ENSG00000281672.1,Gene,ENSG00000281672,1,,80,https://may2015.archive.ensembl.org/Homo_sapie...
135,GRCh38,None,[],87,ENSG00000279857.1,Gene,ENSG00000279857,1,,80,https://may2015.archive.ensembl.org/Homo_sapie...
141,GRCh38,None,[],84,ENSG00000274483.1,Gene,ENSG00000274483,1,,80,https://may2015.archive.ensembl.org/Homo_sapie...


In [21]:
print(versions["permalink"][0])
print(versions["permalink"][25])

https://oct2024.archive.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000151650
https://oct2024.archive.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000142192


In [22]:
# Does every gene have an associated URL?
url_base_len = len(archive_table["url"][0]) + 1
all([len(url) > url_base_len for url in versions["permalink"]])

True

# Part 4: Add permalinks to the gene table

In [23]:
versions = versions[["id", "release", "possible_replacement", "permalink"]]
versions.rename(
    columns={"id": "ensembl_gene_id", "release": "ensembl_release"}, inplace=True
)

gene_table_merged = pd.merge(
    left=gene_table_merged,
    right=versions,
    how="left",
    on="ensembl_gene_id",
    validate="one_to_one",
)

print(gene_table_merged.shape)
gene_table_merged.head()

(35858, 12)


,ensembl_gene_id,_id,_version,alias,name,summary,symbol,type_of_gene,notfound,ensembl_release,possible_replacement,permalink
0,ENSG00000151650,27287,1.0,"[NA88A, HPX42B, VENTX2]",VENT homeobox,This gene encodes a member of the Vent family ...,VENTX,protein-coding,NaN,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
1,ENSG00000168268,64943,1.0,[],5'-nucleotidase domain containing 2,Predicted to enable 5'-nucleotidase activity. ...,NT5DC2,protein-coding,NaN,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
2,ENSG00000186310,4675,1.0,"[MB20, NPL3]",nucleosome assembly protein 1 like 3,This gene is intronless and encodes a member o...,NAP1L3,protein-coding,NaN,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
3,ENSG00000204616,11074,1.0,"[C6orf13, RNF, HCGI, HCG1]",tripartite motif containing 31,This gene encodes a protein that functions as ...,TRIM31,protein-coding,NaN,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
4,ENSG00000158467,23382,1.0,"[IRBIT2, ADOHCYASE3]",adenosylhomocysteinase like 2,The protein encoded by this gene acts as a hom...,AHCYL2,protein-coding,NaN,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...


### Final cleanup
Unfilled "possible_replacement" entries should be changed from NaN to empty lists. 

"possible_replacement" entries that have data in them exist as a list of dicts, and need to have the Ensembl IDs pulled out of them as a list of strings. 

Remove unneeded columns. 

In [24]:
gene_table_merged["possible_replacement"] = gene_table_merged[
    "possible_replacement"
].apply(lambda cell: cell if cell is not np.NaN else [])

gene_table_merged["possible_replacement"] = gene_table_merged.apply(
    lambda row: (
        row["possible_replacement"]
        if len(row["possible_replacement"]) == 0
        else [x["stable_id"] for x in row["possible_replacement"]]
    ),
    axis=1,
)

gene_table_merged = gene_table_merged[
    [
        "ensembl_gene_id",
        "name",
        "alias",
        "summary",
        "symbol",
        "type_of_gene",
        "ensembl_release",
        "possible_replacement",
        "permalink",
    ]
]

gene_table_merged

,ensembl_gene_id,name,alias,summary,symbol,type_of_gene,ensembl_release,possible_replacement,permalink
0,ENSG00000151650,VENT homeobox,"[NA88A, HPX42B, VENTX2]",This gene encodes a member of the Vent family ...,VENTX,protein-coding,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
1,ENSG00000168268,5'-nucleotidase domain containing 2,[],Predicted to enable 5'-nucleotidase activity. ...,NT5DC2,protein-coding,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
2,ENSG00000186310,nucleosome assembly protein 1 like 3,"[MB20, NPL3]",This gene is intronless and encodes a member o...,NAP1L3,protein-coding,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
3,ENSG00000204616,tripartite motif containing 31,"[C6orf13, RNF, HCGI, HCG1]",This gene encodes a protein that functions as ...,TRIM31,protein-coding,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
4,ENSG00000158467,adenosylhomocysteinase like 2,"[IRBIT2, ADOHCYASE3]",The protein encoded by this gene acts as a hom...,AHCYL2,protein-coding,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
...,...,...,...,...,...,...,...,...,...
35853,ENSG00000241472,PTPRG antisense RNA 1,[],NaN,PTPRG-AS1,ncRNA,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
35854,ENSG00000133106,epithelial stromal interaction 1,[BRESI1],The protein encoded by this gene has been show...,EPSTI1,protein-coding,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
35855,ENSG00000230373,"golgin A6 family like 3, pseudogene","[GOLGA6L3, GOLGA6L21P, GOLGA6L17P]",NaN,GOLGA6L3P,pseudo,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...
35856,ENSG00000249738,uncharacterized LOC285626,[LOC105377683],NaN,LOC285626,ncRNA,113,[],https://oct2024.archive.ensembl.org/Homo_sapie...


### Write to a file
This will get uploaded to Synapse as [syn25953363](https://www.synapse.org/#!Synapse:syn25953363).

In [25]:
gene_table_merged = gene_table_merged.sort_values(by="ensembl_gene_id").reset_index(
    drop=True
)
gene_table_merged
gene_table_merged.to_feather("../../output/gene_table_merged_GRCh38.p14.feather")